In [1]:
import argparse
import collections

import numpy as np

import torch
import torch.optim as optim
from torchvision import transforms

from retinanet import model
from retinanet.dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, \
    Normalizer
from torch.utils.data import DataLoader

from retinanet import coco_eval
from retinanet import csv_eval

assert torch.__version__.split('.')[0] == '1'

print('CUDA available: {}'.format(torch.cuda.is_available()))


    

CUDA available: True


In [18]:
args=None
parser = argparse.ArgumentParser(description='Simple training script for training a RetinaNet network.')

parser.add_argument('--dataset', help='Dataset type, must be one of csv or coco.',default='coco')
parser.add_argument('--coco_path',help='Path to COCO directory',default='../patch_testing')
parser.add_argument('--csv_train', help='Path to file containing training annotations (see readme)')
parser.add_argument('--csv_classes', help='Path to file containing class list (see readme)')
parser.add_argument('--csv_val', help='Path to file containing validation annotations (optional, see readme)')

parser.add_argument('--depth', help='Resnet depth, must be one of 18, 34, 50, 101, 152', type=int, default=50)
parser.add_argument('--epochs', help='Number of epochs', type=int, default=100)


_StoreAction(option_strings=['--epochs'], dest='epochs', nargs=None, const=None, default=100, type=<class 'int'>, choices=None, help='Number of epochs', metavar=None)

In [33]:
dataset_train = CocoDataset('/data112/MIDOG2022', set_name='patch_training.json',
                                    transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))
sampler = AspectRatioBasedSampler(dataset_train, batch_size=2, drop_last=False)
dataloader_train = DataLoader(dataset_train, num_workers=3, collate_fn=collater, batch_sampler=sampler)

loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


In [32]:
print(len(dataloader_train))

41104


In [12]:
from pycocotools.coco import COCO
coco= COCO('/data112/MIDOG2022/patch_training.json')

loading annotations into memory...
Done (t=30.34s)
creating index...
index created!


In [50]:
image_ids = coco.getImgIds()
image_index = 1
image_info = coco.loadImgs(image_ids[image_index])[0]

In [51]:
image_info["file_name"]

'1_756_872.png'

In [ ]:
categories = coco.loadCats(coco.getCatIds())
categories.sort(key=lambda x: x['id'])

classes             = {}
coco_labels         = {}
coco_labels_inverse = {}
for c in categories:
    coco_labels[len(self.classes)] = c['id']
    coco_labels_inverse[c['id']] = len(self.classes)
    classes[c['name']] = len(self.classes)

# also load the reverse (label -> name)
labels = {}
for key, value in classes.items():
    labels[value] = key

In [ ]:
def coco_label_to_label(self, coco_label):
        return coco_labels_inverse[coco_label]


In [52]:
annotations_ids = coco.getAnnIds(imgIds=image_ids[image_index], iscrowd=False)
annotations     = np.zeros((0, 5))
coco_annotations = coco.loadAnns(annotations_ids)

In [53]:
for idx, a in enumerate(coco_annotations):

            # some annotations have basically no width / height, skip them
            if a['bbox'][2] < 1 or a['bbox'][3] < 1:
                continue

            annotation        = np.zeros((1, 5))
            annotation[0, :4] = a['bbox']
            annotation[0, 4]  = coco_label_to_label(a['category_id'])
            annotations       = np.append(annotations, annotation, axis=0)

[{'iscrowd': 0,
  'area': 2500,
  'bbox': [231, 231, 50, 50],
  'category_id': 1,
  'image_id': 2,
  'id': 2,
  'scanner_id': 'unknown',
  'tumor_label': 'breast'}]